In [537]:
# New approach
import typing
import copy
import math
import random
import pyvis
import networkx as nx
from pyvis.network import Network as VisualNetwork
"""
XOR using neat
"""

""" 
    1) dokończyć swoją implementację
    2) gymnasium + NEAT
    3) zastąpić sumowanie filtrem kalmana
"""


# activation function on every eval step?

import random

def relu(x):
    if x > 0:
        return x
    else:
        return 0

def sigm(x):
    return 1 / (1 + math.exp(-x))

LOGS = {"WEIGHT_MUTATION":0, "NEW_CONNECTION":0, "NEW_NODE":0, "CROSSOVER":0, "SPECIES":0}

def restart_logs():
    global LOGS
    for x in LOGS:
        LOGS[x] = 0

WEIGHT_MUTATION_CHANCE = 0.8
NEW_CONNECTION_CHANCE = 0.1
NEW_NODE_CHANCE = 0.05
ACTIVATION_FUNC = sigm
C1 = 1.0
C2 = 1.0
C3 = 0.4
SPECIES_THRESHOLD = 3 # Threshold for determining if two genomes belong to the same species

TO_DEBUG = None

In [2]:

class Node:
    def __init__(self, node_id, node_type='hidden'):
        self.node_id = node_id
        self.node_type = node_type  #  'input' | 'hidden' | 'output'

    def __repr__(self):
        return f"{self.node_type} Node {self.node_id} "
    
    def __hash__(self):
        return hash((self.node_id, self.node_type))

    def __eq__(self, other):
        if isinstance(other, Node):
            return self.node_id == other.node_id and self.node_type == other.node_type
        return False

class Connection:
    def __init__(self, in_node, out_node, weight=1.0, enabled=True, innov_number=None):
        self.in_node = in_node
        self.out_node = out_node
        self.weight = weight
        self.enabled = enabled
        self.innov_number = innov_number  # crossover

    def __repr__(self):
        return f"Conn from {self.in_node} to {self.out_node}, weight:{self.weight}, en:{self.enabled}, innv:{self.innov_number}"


In [834]:

class Genome:
    next_innov_number = 0
    next_node_id = 0
    def __init__(self):
        self.nodes = []
        self.connections = []
        self.fitness = 0
   
    def __repr__(self):
        return f"Genome with {len(self.nodes)} nodes and {len(self.connections)} connections. Fitness: {self.fitness}"
    
    @staticmethod
    def construct_from_dict(nodes:dict[int, Node], connections:dict[int,Connection]):
        g = Genome()
        for (i,n) in sorted(nodes.items()):
            #n.node_id = starting_node_id+i
            g.nodes.append(n)
            #Genome.next_node_id = i+1
        
        for (i,c) in sorted(connections.items()):
            if c.innov_number is None:
                c.innov_number = i
                Genome.next_innov_number = i+1
            g.connections.append(c)
        return g
    

    def set_fitness(self, new):
        self.fitness = new

    @staticmethod
    def create_from_shape(shape:tuple[int,int]):
        inputs,outputs = shape
        g = Genome()

        for i in range(inputs):
            g.add_node('input')

        for o in range(outputs):
            g.add_node('output')

        ins = list(filter(lambda x:x.node_type == 'input', g.nodes))
        outs = list(filter(lambda x:x.node_type == 'output', g.nodes))
        for i in ins:
            for o in outs:
                g.add_connection(i.node_id, o.node_id, 1.0)
        return g

    def add_node(self, node_type):
        node = Node(Genome.next_node_id, node_type)
        self.nodes.append(node)
        Genome.next_node_id = Genome.next_node_id + 1
        return node

    def evaluate(self, input):
        ff_in = {}
        for (i, n) in enumerate(input):
            ff_in[i] = n
        return feed_forward(self, ff_in, ACTIVATION_FUNC)

    def add_connection(self, in_node, out_node, weight, mark=None):
        connection = Connection(in_node, out_node, weight, innov_number=Genome.next_innov_number)
        if mark is not None:
            connection.mark = mark
        self.connections.append(connection)
        Genome.next_innov_number += 1
    
    def show(self):
        graph = nx.Graph()
        nt = VisualNetwork(notebook=True , cdn_resources='in_line',layout=True, directed=True)
        for n in self.nodes:nt.add_node(
            n.node_id,
            label=f"{n.node_type},{n.node_id}",
            level = 6 if n.node_type == 'input' else 0 if n.node_type == 'output' else random.randint(1,5),
            )
        for c in self.connections:
            nt.add_edge(source=c.in_node,to=c.out_node, title=f"in:{c.innov_number}\nw:{c.weight}", hidden=(not c.enabled) )
        nt.from_nx(graph) 

        nt.show('example.html')
        !open "example.html"

    def mutate(self):
        for connection in self.connections:
            if random.random() < WEIGHT_MUTATION_CHANCE:
                print("WEIGHT_MUTATION")
                LOGS["WEIGHT_MUTATION"] += 1
                connection.weight += random.uniform(-1, 1) 

        if random.random() < NEW_CONNECTION_CHANCE:
            node1 = random.choice(self.nodes)
            node2 = random.choice(self.nodes)
            if node1.node_type != 'output' and\
                node2.node_type != 'input' and\
                node1.node_id != node2.node_id and\
                not any([c for c in self.connections if ((c.in_node == node1.node_id and c.out_node == node2.node_id and c.enabled) or (c.in_node == node2.node_id and c.out_node == node1.node_id and c.enabled)) ]):

                print(f"NEW CONNECTION {node1.node_id} -> {node2.node_id}")
                LOGS["NEW_CONNECTION"] += 1
                self.add_connection(node1.node_id, node2.node_id, random.uniform(-1, 1))

        if random.random() < NEW_NODE_CHANCE:
            print(f"BEFORE NEW NODE:, Genome.next_node_id:{Genome.next_node_id}, Next innov:{Genome.next_innov_number}")
            connection = random.choice(self.connections)
            connection.enabled = False
            print("NEW NODE")
            LOGS["NEW_NODE"] += 1
            new_node = self.add_node('hidden')
            self.add_connection(connection.in_node, new_node.node_id, 1.0, mark=f"NNC1 {connection} gen:{GENERATION}")
            self.add_connection(new_node.node_id, connection.out_node, connection.weight, mark=f"NNC2 {connection} gen:{GENERATION}")
            print(f"AFTER NEW NODE:, Genome.next_node_id:{Genome.next_node_id}, Next innov:{Genome.next_innov_number}")

    def compatibility_distance(self, other_genome):
        matching_weight_diffs = []
        disjoint_count = 0
        excess_count = 0

        self_genes = {c.innov_number: c for c in self.connections}
        other_genes = {c.innov_number: c for c in other_genome.connections}

        for innov in set(self_genes.keys()).union(other_genes.keys()):
            if innov in self_genes and innov in other_genes:
                matching_weight_diffs.append(abs(self_genes[innov].weight - other_genes[innov].weight))
            elif innov in self_genes:
                if innov > max(other_genes.keys()):
                    excess_count += 1
                else:
                    disjoint_count += 1
            elif innov in other_genes:
                if innov > max(self_genes.keys()):
                    excess_count += 1
                else:
                    disjoint_count += 1

        avg_weight_diff = sum(matching_weight_diffs) / len(matching_weight_diffs) if matching_weight_diffs else 0.0
        N = max(len(self.connections), len(other_genome.connections))
        N = max(N, 1)

        distance = (C1 * disjoint_count + C2 * excess_count) / N + C3 * avg_weight_diff
        return distance

In [5]:
def crossover(genome1:Genome, genome2:Genome):

    def equals(g1, g2):
        return abs(g1.fitness - g2.fitness) < 0.01

    def calculate_weight_for_common(g1:Connection, g2:Connection, method='avg'):
        "Return either function for averaging weights or picking one randomly from genomes"
        assert (method == 'avg' or method=='rand')
        if method == 'avg':
            return (g1.weight + g2.weight ) / 2
        else:
            return g1.weight if random.random() > 0.5 else g2.weight

    gene_comparison = {}
    if genome2.fitness > genome1.fitness:
        tmp = genome1
        genome1 = genome2
        genome2 = tmp
    for (conn, fromGene) in list(zip(genome1.connections, len(genome1.connections)*[0])) + list(zip(genome2.connections, len(genome2.connections) * [1])):
        present = gene_comparison.get(conn.innov_number)
        if present is not None:
            present[fromGene] = conn
        else:
            gene_comparison[conn.innov_number] = { fromGene : conn } 

    are_equal = equals(genome1, genome2)
    sorted_keys = sorted(gene_comparison.keys())
    product = []
    highest_innov_for_shorter_parent = min([ max([g.innov_number for g in genome1.connections]), max([g.innov_number for g in genome2.connections]) ])
    for gene_i in sorted_keys:
        pair = gene_comparison[gene_i]
        g0, g1 = pair.get(0), pair.get(1)
        if g0 is not None and g1 is not None and g0.innov_number == g1.innov_number:
            tmp = copy.deepcopy(g0)
            tmp.weight = calculate_weight_for_common(g0, g1)
            product.append(tmp)
            continue
        if are_equal:
            assert are_equal, "Fitness of both genomes is equal, picking genes at random"
            if g0 is None or g1 is None:
                present = g0 or g1
                if present.innov_number > highest_innov_for_shorter_parent:
                    "Excess"
                    if random.random() > 0.5:
                        "equal - get inherited by random parent"
                        product.append(copy.deepcopy(present))
                        # not equal
                else:
                    assert (not (g1 is not None and g0 is not None)), "Disjoint, collect at random"
                    choice = pair.get(round(random.random())) # 0 or 1
                    if choice is not None:
                        product.append(copy.deepcopy(choice))
        else:
            assert not are_equal, "Fitness of both genomes is not equal, picking only options from best one"
            if g0 is not None:
                product.append(copy.deepcopy(g0))
                #disjoint - gets inherited from fitter parent, or from random if they're equal
    # ok, lecimy: Node'y
    ins_and_outs = set()
    for c in product:
        ins_and_outs.add(c.in_node)
        ins_and_outs.add(c.out_node)
    nodes = list(filter(lambda n: n.node_id in ins_and_outs, set(genome1.nodes).union(set(genome2.nodes))))
    ret = Genome.construct_from_dict(nodes={n.node_id:n for n in nodes}, connections={ p.innov_number:p for p in product})
    print("CROSSOVER")
    LOGS["CROSSOVER"] += 1
    ret.mark = f"Crossover{GENERATION}"
    #ret.parents = [genome1, genome2]
    return ret


In [6]:

class Species:
    def __init__(self, representative):
        self.members = [representative]
        self.representative = representative

    def add_member(self, genome):
        self.members.append(genome)

    def clear_members(self):
        self.members = []

    def adjust_fitnesses(self):
        total_fitness = sum([genome.fitness for genome in self.members])
        for genome in self.members:
            genome.fitness /= total_fitness

    def set_representative(self):
        self.representative = random.choice(self.members)


def speciate(genomes, species):
    for spec in species:
        spec.clear_members()

    for genome in genomes:
        found_species = False
        for spec in species:
            if genome.compatibility_distance(spec.representative) < SPECIES_THRESHOLD:
                spec.add_member(genome)
                found_species = True
                LOGS["SPECIES"] += 1
                break
        if not found_species:
            new_species = Species(genome)
            species.append(new_species)

    # Remove empty species
    species = [spec for spec in species if spec.members]

    return species


In [731]:

import numpy as np
def feed_forward(genome:Genome, input_values:dict[int,float], activation_func):
    try:
        DEFAULT_BIAS_VALUE = 1.0
        """
        O boze o kurwa
        To moze mieć rekurencyjne połączenia
        TODO zaimplementować to.
        """
        all_output_nodes = set([n for n in genome.nodes if n.node_type == 'output'])
        valid_connections = [ c for c in genome.connections if c.enabled] 
        dependency_tree = {}
        values = input_values 
        all_output_nodes = set([n for n in genome.nodes if n.node_type == 'output'])
        # allow for biases to appear (if node is not input and has no incoming connections)
        values.update({n.node_id: DEFAULT_BIAS_VALUE for n in genome.nodes if n.node_type == 'hidden' and not any([c.out_node == n.node_id for c in valid_connections])})
        for c in valid_connections:
            if dependency_tree.get(c.out_node) is not None:
                dependency_tree[c.out_node].add(c)
            else:
                dependency_tree[c.out_node] = set([c])

        def eval_node(node_id):
            if(values.get(node_id) is not None):
                return values.get(node_id)
            else:
                deps = []
                dependants = dependency_tree.get(node_id)
                for conn in dependants:
                    deps.append(eval_node(conn.in_node) * conn.weight)
                v = activation_func(sum(deps))
                values[node_id] = v
                return v
        final = [eval_node(o.node_id) for o in all_output_nodes]
        if len(final) == 0:
            print(f"genome: {genome} ")
            #genome.show()
            return final
        else:
            return final
    except Exception as e:
        global TO_DEBUG
        TO_DEBUG = genome
        raise e

In [809]:
TO_DEBUG.connections



[Conn from 0 to 2, weight:-1.4362788240081585, en:True, innv:1,
 Conn from 1 to 2, weight:0.024503715169874374, en:True, innv:2,
 Conn from 3 to 2, weight:-1.628345334555162, en:True, innv:3,
 Conn from 0 to 4, weight:0.4962756776287449, en:False, innv:4,
 Conn from 1 to 4, weight:0.7979104178833114, en:True, innv:5,
 Conn from 4 to 2, weight:-0.20350792353160796, en:True, innv:6,
 Conn from 0 to 2, weight:1.0, en:True, innv:4,
 Conn from 2 to 4, weight:0.4962756776287449, en:True, innv:5]

In [711]:
INPUT = [[0,0], [0,1], [1,0], [1,1]]
OUTPUT = [0, 1, 1, 0]
import copy
def eval(genome):
    """
    Evaluation function as specified in the paper. TODO: research why exactly is it like that.
    """
    #result = [feed_forward(genome, input_values= dict(enumerate(iv)), activation_func=ACTIVATION_FUNC)[0] for iv in INPUT]
    result = []
    for i, iv in enumerate(INPUT):
        res= feed_forward(genome, input_values= dict(enumerate(iv)), activation_func=ACTIVATION_FUNC) 
        print(f"input: {iv}, output: {res}, truth: {OUTPUT[i]}")
        result.append(res[0])
    assert(len(OUTPUT) == len(result))
    error = sum([ abs(r - o) for (r,o) in zip(result, OUTPUT)])
    y = (4 - error) ** 2
    return y


In [851]:
# aight, lets artificially construct the 'good' solution to check if my eval will even work
g1 = Genome.construct_from_dict(
        nodes={
            0:Node(0, 'input'),
            1:Node(1, 'input'),
            2:Node(2, 'output'),
            3:Node(3, 'hidden'),
            4:Node(4, 'hidden'),
        },
        connections={
            0:Connection(in_node=0, out_node=2),
            1:Connection(in_node=1, out_node=2),
            2:Connection(in_node=3, out_node=2),
            3:Connection(in_node=0, out_node=4),
            4:Connection(in_node=1, out_node=4),
            5:Connection(in_node=4, out_node=2),
        }
    )
#g1.show()
eval(g1)

input: [0, 0], output: [0.8175744761936437], truth: 0
input: [0, 1], output: [0.9388346534008736], truth: 1
input: [1, 0], output: [0.9388346534008736], truth: 1
input: [1, 1], output: [0.9797827978364861], truth: 0


4.32769815369438

In [695]:
feed_forward(g1, {0:1, 1:0}, sigm)

genome: Genome with 4 nodes and 6 connections. Fitness: 0 


[]

In [724]:
g1.show()

example.html
64494.33s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [792]:
g1.nodes

[input Node 0 , input Node 1 , output Node 2 , hidden Node 3 , hidden Node 4 ]

In [793]:
g1.show()

example.html
65849.30s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [790]:
Genome.next_innov_number = 0
Genome.next_node_id = 0

In [800]:
g1.mutate()

WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION


In [801]:
eval(g1)

input: [0, 0], output: [0.8921564362394694], truth: 0
input: [0, 1], output: [0.981741088376044], truth: 1
input: [1, 0], output: [0.9104689601669768], truth: 1
input: [1, 1], output: [0.9798942442375935], truth: 0


4.08104387238465

In [683]:

candidate = sorted(p.genomes,key=lambda x: x.fitness, reverse=True)[0]
eval(candidate)
candidate.show()

input: [0, 0], output: [0.5], truth: 0
input: [0, 1], output: [0.6988314359277137], truth: 1
input: [1, 0], output: [0.6793954573960537], truth: 1
input: [1, 1], output: [0.83100046650024], truth: 0
example.html
63391.08s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [852]:
GENERATION = 0
class Population:
    """
    Toplevel object
    """
    population = 40
    max_iters = 100
    percentile_best_from_previous_population_to_transfer = 0.1
    percentile_to_reproduce = 0.5
    probability_crossover = 0.3
    probability_mutation_after_crossover = 0.3
    def __init__(self, default, min_success_score=1, evaluation_function=eval):
        global GENERATION
        GENERATION = 0
        self.genomes = []
        Genome.next_innov_number = 0
        Genome.next_node_id = 0
        #default = Genome.create_from_shape(shape)
        default.fitness = 1
        for _ in range(self.population):
            n = copy.deepcopy(default)
            for c in n.connections:
                c.weight= random.uniform(-1,1)
            self.genomes.append(n)
        self.species = [Species(self.genomes[0])]
        Genome.next_innov_number = len(default.connections)
        Genome.next_node_id = len(default.nodes)

    def show_generation(self):
        #print amount of species, and amount of genomes in each species. print top 5 genomes in each species
        print('#############################################')
        print(f"***Generation {GENERATION}***")
        print(f"Species: {len(self.species)}")
        for (i,s) in enumerate(self.species):
            print(f"Species {i} has {len(s.members)} members, top 5 are {sorted(s.members, reverse=True, key=lambda g: g.fitness)[0:5]}")
        print(f"Top 5 genomes are {sorted(self.genomes, reverse=True, key=lambda g: g.fitness)[0:5]}")

    
    def evolve(self) -> list[Genome]:
        distribution = [ sum([g.fitness for g in s.members]) for s in self.species]
        # eliminate lowest performing specimen in each specie
        for s in self.species:
            s.members = sorted(s.members, reverse=True, key=lambda g: g.fitness)[0:max(int(len(s.members) * self.percentile_to_reproduce),1)]

        new_population = [copy.deepcopy(g) for g in self.genomes[0:(int(len(self.genomes) * self.percentile_best_from_previous_population_to_transfer))]]
        while len(new_population) < self.population:
            to_select_for_reproduction = random.choices(self.species, weights=distribution, k=1)[0]
            if random.random() < self.probability_crossover:
                #p1 = random.choice(to_select_for_reproduction.members)
                #p2 = random.choice(self.genomes)
                p1,p2 = random.choices(self.species, weights=distribution, k=2)
                p1= random.choice(p1.members)
                p2= random.choice(p2.members)

                child = crossover(p1, p2)
                if random.random() < self.probability_mutation_after_crossover:
                    child.mutate()
                new_population.append(child)
            else:
                g = copy.deepcopy(random.choice(to_select_for_reproduction.members))
                g.mutate()
                new_population.append(g)
        return new_population
    

    def run(self):
        global GENERATION
        # Create next generation
        self.genomes = self.evolve()
        # Partition new generation into species
        self.species = speciate(self.genomes, self.species)
        # Evaluate fitness
        GENERATION += 1
        results = map(eval, self.genomes)
        for (g, v) in zip(self.genomes, results): g.fitness = v 
        #for s in self.species: s.adjust_fitnesses()
        self.genomes = sorted(self.genomes, reverse=True, key=lambda g: g.fitness)
        self.show_generation()
        for s in self.species: s.adjust_fitnesses()
        self.genomes = sorted(self.genomes, reverse=True, key=lambda g: g.fitness)
        # Check criteria

        # if self.min_success_score <= self.genomes[0].fitness:
        #     print("Score threshold exceeded. Finishing...")
        #     return True


    def run_all(self):
        while self.generation < self.max_iters: 
            self.run()


In [2276]:
len(p.species[0].members)


40

In [861]:

p = Population(g1)
TO_DEBUG = None
restart_logs()
p.run()



CROSSOVER
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
CROSSOVER
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
CROSSOVER
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
CROSSOVER
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
BEFORE NEW NODE:, Genome.next_node_id:5, Next innov:6
NEW NODE
AFTER NEW NODE:, Genome.next_node_id:6, Next innov:8
CROSSOVER
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WE

In [845]:
tmp = p.genomes[0]


In [838]:
tmp.show()

example.html
67530.96s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [854]:
tmp.connections

[Conn from 0 to 2, weight:1.3418897304104571, en:True, innv:1,
 Conn from 1 to 2, weight:0.9253856473180779, en:True, innv:2,
 Conn from 3 to 2, weight:0.03540894421099794, en:True, innv:3,
 Conn from 0 to 4, weight:-0.05682695273361982, en:True, innv:4,
 Conn from 1 to 4, weight:-0.46552686654769504, en:True, innv:5,
 Conn from 4 to 2, weight:0.2542184020273037, en:True, innv:6]

In [869]:
g1.connections

[Conn from 0 to 2, weight:1.0, en:True, innv:0,
 Conn from 1 to 2, weight:1.0, en:True, innv:1,
 Conn from 3 to 2, weight:1.0, en:True, innv:2,
 Conn from 0 to 4, weight:1.0, en:True, innv:3,
 Conn from 1 to 4, weight:1.0, en:True, innv:4,
 Conn from 4 to 2, weight:1.0, en:True, innv:5]

In [848]:
tmp.nodes

[input Node 0 , input Node 1 , output Node 2 , hidden Node 3 , hidden Node 4 ]

In [831]:
g1.nodes

[input Node 0 , input Node 1 , output Node 2 , hidden Node 3 , hidden Node 4 ]

In [868]:
Genome.next_innov_number

12

In [870]:
Genome.next_node_id

8

In [804]:
for i in range(20):
    p.run()

WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
CROSSOVER
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
CROSSOVER
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
CROSSOVER
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUT

RecursionError: maximum recursion depth exceeded

In [650]:
p.species[1].members[0].show()

example.html
62338.84s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [807]:


TO_DEBUG.connections




[Conn from 0 to 2, weight:-1.4362788240081585, en:True, innv:1,
 Conn from 1 to 2, weight:0.024503715169874374, en:True, innv:2,
 Conn from 3 to 2, weight:-1.628345334555162, en:True, innv:3,
 Conn from 0 to 4, weight:0.4962756776287449, en:False, innv:4,
 Conn from 1 to 4, weight:0.7979104178833114, en:True, innv:5,
 Conn from 4 to 2, weight:-0.20350792353160796, en:True, innv:6,
 Conn from 0 to 2, weight:1.0, en:True, innv:4,
 Conn from 2 to 4, weight:0.4962756776287449, en:True, innv:5]

In [787]:
TO_DEBUG.connections[-1]

Conn from 2 to 4, weight:-0.4445488079575617, en:True, innv:5

In [808]:
TO_DEBUG.nodes

[input Node 0 ,
 input Node 1 ,
 output Node 2 ,
 hidden Node 3 ,
 hidden Node 4 ,
 hidden Node 2 ]

In [2287]:
tmp = [pg for pg in p.genomes if len(pg.nodes) > (sum([ len(g.nodes) for g in p.genomes] ) / len(p.genomes))]
tmp[0].show()

example.html
169319.48s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [904]:



for g in p.genomes: g.show()




example.html
example.html
example.html
example.html
example.html


In [2254]:
for gpp in p.genomes[:5]: gpp.show()


example.html
example.html
example.html
example.html
example.html


In [648]:
LOGS

{'WEIGHT_MUTATION': 9961,
 'NEW_CONNECTION': 17,
 'NEW_NODE': 203,
 'CROSSOVER': 1488,
 'SPECIES': 5638}

In [2224]:
p.run()
#TODO After a couple runs assertionerror throws `dependants is not None`. Fix that

WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
WEIGHT_MUTATION
input: [0, 0], output: [0.5], truth: 0
input: [0, 1], output: [1.0], truth: 1
input: [1, 0], output: [1.0], truth: 1
input: [1, 1], output: [1.0], truth: 0
input: [0, 0], output: [0.5], truth: 0
input: [0, 1], output: [1.0], truth: 1
input: [1, 0], output: [1.0], truth: 1
input: [1, 1], output: [1.0], truth: 0
input: [0, 0], output: [0.5], truth: 0
input: [0, 1], output: [1.0], truth: 1
input: [1, 0], output: [1.0], truth: 1
input: [1, 1], output: [1.0], truth: 0
input: [0, 0], output: [0.5], truth: 0
input: [0, 1], output: [1.0], truth: 1
input: [1, 0], output: [1.0], truth: 1
input: [1, 1], output: [1.0], truth: 0
input: [0, 0], output: [0.5], truth: 0
input: [0, 1], output: [1.0], truth: 1
input: [1, 0], output: [1.0], truth: 1
input: [1, 1], output: [1.0], truth: 0
Generation 287
Species: 1
Species 0 has 5 members, top 5 are [Genome with 3 

# TODO: FIX THIS UP HERE

In [1294]:
TO_DEBUG.show()

example.html


AttributeError: 'Genome' object has no attribute 'mark'

In [12]:
len(p.species[0].members)

40

In [57]:
def test_crossover():
    g1 = Genome.construct_from_dict(
            nodes={
                1:Node(1, 'input'),
                2:Node(2, 'input'),
                3:Node(3, 'input'),
                4:Node(4, 'output'),
                5:Node(5),
            },
            connections={
                1:Connection(in_node=1, out_node=4),
                2:Connection(in_node=2, out_node=4, enabled=False),
                3:Connection(in_node=3, out_node=4),
                4:Connection(in_node=2, out_node=5),
                5:Connection(in_node=5, out_node=4 ),
                8:Connection(in_node=1, out_node=5),
            }
        )
    g2 = Genome.construct_from_dict(
            nodes={
                1:Node(1, 'input'),
                2:Node(2, 'input'),
                3:Node(3, 'input'),
                4:Node(4, 'output'),
                5:Node(5),
                6:Node(6),
            },
            connections= {
                1:Connection(in_node=1, out_node=4),
                2:Connection(in_node=2, out_node=4, enabled=False),
                3:Connection(in_node=3, out_node=4),
                4:Connection(in_node=2, out_node=5),
                5:Connection(in_node=5, out_node=4, enabled=False),
                6:Connection(in_node=5, out_node=6),
                7:Connection(in_node=6, out_node=4),
                9:Connection(in_node=3, out_node=5),
                10:Connection(in_node=1, out_node=6),
            }
        )
    #g1.show()
    #g2.show()
    g1.set_fitness(0.5)
    g2.set_fitness(0.5)
    c = crossover(g1,g2)
    c.show()
#test_crossover()
    

In [ ]:
class Problem:
    def __init__(self, inputs, outputs, compute_fitness):
        if len(inputs) != len(outputs): raise Exception("Number of inputs and outputs dont match")


In [ ]:
# One of the first tests, checking if structures were building correctly
gen = Genome()
gen.add_node('input') #     0
gen.add_node('input') #     1
gen.add_node('hidden')#     2
gen.add_node('output')#     3
gen.add_node('output')#     4
gen.add_connection(0,4, weight=0.8)
gen.add_connection(1,2, weight=0.8)
gen.add_connection(0,3, weight=0.8)
gen.add_connection(1,4, weight=0.8)
gen.add_connection(2,3, weight=0.8)
result = feed_forward(gen, {0:0.25, 1:0.5}, relu)
print("RESULT:", result)